In [ ]:
pip install pandas_datareader

In [ ]:
%load_ext autoreload
%autoreload 2

import stock_portfolio as sp
len(sp.stickers)

In [ ]:
#sp.fetch_all_price()
sp.fetch_price('D05.SI')

In [ ]:
p = sp.get_price('0700.HK')
print(p)

In [ ]:
ledger = sp.get_ledger()
ledger

In [ ]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

ledger['cost'] = ledger['quantity']*ledger['price']+ledger['commission']

buy_ledger = ledger[ledger['type']=='buy']
sell_ledger = ledger[ledger['type']=='sell']
buy_quantity = buy_ledger.groupby(['tick'])['quantity'].sum()
sell_quantity = sell_ledger.groupby(['tick'])['quantity'].sum()

positions = pd.DataFrame(data=(buy_quantity.sub(sell_quantity, fill_value=0).fillna(0)))
positions['cost'] =buy_ledger.groupby(['tick'])['cost'].sum()
positions['avg price'] = positions['cost']/buy_quantity

positions['currency'] = 'USD'
positions['last price'] = 0
for index, row in positions.iterrows():
    if index.endswith('.HK'):
        positions.at[index, 'currency'] = 'HKD'
    elif index.endswith('.SI'):
        positions.at[index, 'currency'] = 'SGD'
    elif index.endswith('.SS'):
        positions.at[index, 'currency'] = 'CNY'
    
    positions.at[index, 'last price'] = sp.get_price(index).iloc[-1]['Adj Close']

positions['cur value'] = positions['quantity'] * positions['last price']
positions['unreal p&l'] = positions['cur value'] - positions['cost']
positions['return'] = positions['unreal p&l'] / positions['cost']
positions